In [1]:
import numpy as np

MAC = 512

def hex_to_number(hex):
    number = int(hex, 16)
    if(number>127):
        number = number - 256
    return number

def binary_to_number(binary):
    number = int(binary, 2)
    if(number>8388607):
        number = number - 16777216
    return number

def read_result_mem(fileName = "data/result_mem.csv", row=1024, col=512):
    shape = (row, col)
    array = np.zeros(shape).astype(np.int32)
    f = open(fileName, "r")
    for i in range(row):
        for j in range(col//8):# 512/8
            for k in range(8):
                tmpc = f.read(2)
                array[i][8*j+k] = hex_to_number(tmpc)
                # print(tmpc)
                if k==7:
                    if f.read(1) != '\n':
                        print("read error")
    # print('reading matrix:\n',array)
    return array

def read_result_mem_final(fileName = "data/result_mem.csv", row=512, col=512):
    shape = (row, col)
    array = np.zeros(shape).astype(np.int32)
    f = open(fileName, "r")
    for rows in range(4):
        for cols in range(4):
            for i in range(row//4):
                for j in range(col//8):
                    for k in range(2):
                        f.read(8)
                        tmpc = f.read(24)
                        array[rows*(row//4)+i][2*j+k+cols*(col//4)] = binary_to_number(tmpc)
                        if k==1:
                            if f.read(1) != '\n':
                                print("read error")
    # print('reading matrix:\n',array)
    return array


# main entry
def main():
    # get original input matrix
    array = read_result_mem(fileName = "data/input_mem.csv",row=MAC*2, col=MAC)
    #array0 = np.load("in_1.npy")
    #if np.array_equal(array, array0):
        #print("Correct!")
    a1 = array[0:MAC,:]
    a2 = array[MAC:MAC*2,:]

    # get computed results from Verilog
    correct_result = np.matrix(a1).astype(int)*np.matrix(a2).astype(int)
    print(f"a1:\n{a1}\na2:\n{a2}\ncorrect_result:\n{correct_result}")

    # check results is correct
    result_array = read_result_mem_final(fileName = "data/result_mem.csv",row=MAC, col=MAC)
    np.save("data/result.npy", result_array)
    for i in range(MAC):
        for j in range(MAC):
            if(result_array[i,j] != correct_result[i,j]):
                print(f"error at {i},{j} correct:{correct_result[i,j]} result:{result_array[i,j]}")
    # result_array = read_result_mem(fileName = "result_mem.csv",row=512)
        
    loss = np.sum(np.square(correct_result-result_array)) #mean-square error
    relative_loss = np.sum(np.square(correct_result-result_array))/np.sum(np.square(correct_result)) #relative mean-square error
    print(f">>loss is {loss}\n>>relative_loss is {relative_loss}")

if __name__=="__main__": 
    main()


a1:
[[  71    0  -37 ...  -57    0   88]
 [  93    0  124 ...  110  -41  -50]
 [   0   98  -73 ...   32    0  -99]
 ...
 [  94    0   68 ...    0    0  -20]
 [ -78  -26  -32 ...    0   90    0]
 [   0 -114   38 ...   40    0    0]]
a2:
[[  91   49  -67 ...    0   40  -11]
 [   4    0   -5 ...   94  106 -122]
 [-127 -121    0 ... -103    0 -118]
 ...
 [ -98 -116  -20 ...  -99    0  -32]
 [   0   64  -82 ...  126   47 -122]
 [ -59    0    0 ...    0  -45  104]]
correct_result:
[[ -73208   57396  -73310 ...  -21458   22199   28327]
 [ -38260  -13976  -37723 ...   41037  -11066  -29706]
 [  66928  167854   55077 ...   87633   72949   40964]
 ...
 [ 101277  -97538  -25929 ... -120845   55233  126257]
 [  36193 -112564   68742 ...   78678  143687  -83287]
 [ -41111  -27481    4031 ...  101090 -102723   48675]]
>>loss is 0
>>relative_loss is 0.0
